# Limpieza de dataset de Características de las localidades y del entorno urbano 2014
## 1. Introduccion

Indicadores que salen de deste dataset: 

ID |Descripción
---|:----------
P0702|Restricción al paso de automóviles
P0703|Restricción al paso a peatones
P0704|Vialidades con recubrimiento de la calle
P0705|Vialidades con banqueta
P0706|Vialidades con guarnición
P0713|vialidades con rampa para silla de ruedas



In [2]:
descripciones = {
'P0702' : 'Restricción al paso de automóviles',
'P0703' : 'Restricción al paso a peatones',
'P0704' : 'Vialidades con recubrimiento de la calle',
'P0705' : 'Vialidades con banqueta',
'P0706' : 'Vialidades con guarnición',
'P0713' : 'vialidades con rampa para silla de ruedas',
}

In [3]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import csv
import zipfile

In [4]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


In [5]:
# URL Fuente
remote_path = r'http://www.beta.inegi.org.mx/contenidos/proyectos/encotras/cleu/2014/tabulados/tab_entorno_2014_xls.zip'

# Carpeta destino Local
local_path = r'D:\PCCS\00_RawData\01_CSV\CLEU\tab_entorno_2014_xls.zip'

## Descarga de datos
Todos los datos se encuentran en un solo archivo

In [7]:
# Descarga de archivo
if os.path.isfile(local_path):
    print('Ya existe el archivo: {}'.format(local_path))
else:
    print('Descargando {} ... ... ... ... ... '.format(local_path))
    urllib.request.urlretrieve(remote_path, local_path) #
    print('se descargó {}'.format(local_path))


Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CLEU\tab_entorno_2014_xls.zip


In [13]:
#Descompresión de archivo
target = r'D:\PCCS\00_RawData\01_CSV\CLEU'
archivo = target + '\\' + 'tab_entorno_2014.xls'

if os.path.isfile(target + '\\' + 'tab_entorno_2014.xls'):
    print('Ya existe el archivo: {}'.format(archivo))
else:
    print('Descomprimiendo {} ... ... ... ... ... '.format(target))
    descomprimir = zipfile.ZipFile(local_path, 'r')
    descomprimir.extractall(target)
    descomprimir.close

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\CLEU\tab_entorno_2014.xls


## Estandarizacion del dataset

In [15]:
# Cada parametro se encuentra en una hoja diferente del libro de excel
# Ubicacion de datos del cada parametro:
hojas = {
    'P0702': 'TEU07', # Restricción al paso de automóviles
    'P0703': 'TEU07', # Restricción al paso a peatones
    'P0704': 'TEU12', # Vialidades con recubrimiento de la calle
    'P0705': 'TEU02', # Vialidades con banqueta
    'P0706': 'TEU03', # Vialidades con guarnición
    'P0713': 'TEU04' # vialidades con rampa para silla de ruedas
}

Cada hoja tiene columnas diferentes, a continuación se definen para pasar los nombres durante el proceso de lectura de hojas:

In [19]:
# Nombre de las hojas a partir de su descripcion en el archivo fuente
nom_hojas = {
    'P0702': 'Manzanas por entidad federativa, municipio, localidad y restricción del paso a automóviles según restricción del paso a peatones en sus vialidades', # Restricción al paso de automóviles
    'P0703': 'Manzanas por entidad federativa, municipio, localidad y restricción del paso a automóviles según restricción del paso a peatones en sus vialidades', # Restricción al paso a peatones
    'P0704': 'Vialidades por entidad federativa, municipio, localidad y alumbrado público según tipo de recubrimiento de la calle', # Vialidades con recubrimiento de la calle
    'P0705': 'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de recubrimiento de la calle', # Vialidades con banqueta
    'P0706': 'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de guarnición en sus vialidades', # Vialidades con guarnición
    'P0713': 'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de rampa para silla de ruedas en sus vialidades' # vialidades con rampa para silla de ruedas
}

Debido a que se saltarán renglones al momento de leer cada hoja, será necesario renombrar todas las columnas para cada hoja. De una vez, como en el archivo original todas las columnas tienen nombres similares sin imporatar el tema, en el renombrado de columnas se incluirá el tema al que aluda cada dataset.

In [40]:
dic_hojas = {
    'TEU07': [     # Restricción al paso de automóviles
        'Clave geoestadística',
        'Entidad federativa',
        'Municipio',
        'Localidad',
        'Restricción del paso a automóviles',
        'Total de manzanas',
        'RPPeatones Todas las vialidades',
        'RPPeatones Alguna vialidad',
        'RPPeatones Ninguna vialidad',
        'RPPeatones No especificado'
    ],
    'TEU12': [     # Vialidades con recubrimiento de la calle
        'Clave geoestadística',
        'Entidad federativa',
        'Municipio',
        'Localidad',
        'Disponibilidad de alumbrado público',
        'Total de vialidades',
        'Pavimento o concreto',
        'Empedrado o adoquín',
        'Sin recubrimiento',
        'Recubrimiento No especificado',
    ], 
    'TEU02': [     # Vialidades con banqueta
        'Clave geoestadística',
        'Entidad federativa',
        'Municipio',
        'Localidad',
        'Disponibilidad de banqueta',
        'Total de manzanas',
        'Banqueta en todas las vialidades',
        'Banqueta en Alguna vialidad',
        'Banqueta en Ninguna vialidad',
        'Banqueta No especificado'
    ], 
    'TEU03': [     # Vialidades con guarnición
        'Clave geoestadística',
        'Entidad federativa',
        'Municipio',
        'Localidad',
        'Disponibilidad de banqueta',
        'Total de manzanas',
        'Guarnicion en Todas las vialidades',
        'Guarnicion en Alguna vialidad',
        'Guarnicion en Ninguna vialidad',
        'Guarnicion No especificado'
    ],
    'TEU04': [     # vialidades con rampa para silla de ruedas
        'Clave geoestadística',
        'Entidad federativa',
        'Municipio',
        'Localidad',
        'Disponibilidad de banqueta',
        'Total de manzanas',
        'Rampa en Todas las vialidades',
        'Rampa en Alguna vialidad',
        'Rampa en Ninguna vialidad',
        'Rampa No especificado'
    ]
}


In [23]:
# Numero de filas que se tienen que ignorar en cada hoja antes de llegar a los datos.
skip = 8

In [41]:
# Funcion para leer datos de hojas
def cargahoja(hoja, colnames, skip):
    raw_data = pd.read_excel(archivo,
                              sheetname=hoja,
                              skiprows=skip).dropna()
    # renombra las columnas
    raw_data.columns = colnames
    return raw_data

In [77]:
datasets = {}
for parametro, hoja in hojas.items():
    print('Procesando {} para {}'.format(hoja, parametro))
    datasets[parametro] = cargahoja(hoja, dic_hojas[hoja], skip)
    print('------terminado------')

Procesando TEU07 para P0702
------terminado------
Procesando TEU07 para P0703
------terminado------
Procesando TEU12 para P0704
------terminado------
Procesando TEU02 para P0705
------terminado------
Procesando TEU03 para P0706
------terminado------
Procesando TEU04 para P0713
------terminado------


In [51]:
datasets.keys()

dict_keys(['P0702', 'P0703', 'P0704', 'P0705', 'P0706', 'P0713'])

In [49]:
descripciones

{'P0702': 'Restricción al paso de automóviles',
 'P0703': 'Restricción al paso a peatones',
 'P0704': 'Vialidades con recubrimiento de la calle',
 'P0705': 'Vialidades con banqueta',
 'P0706': 'Vialidades con guarnición',
 'P0713': 'vialidades con rampa para silla de ruedas'}

## Limpieza de datasets
Hay algunos procesos estandar que se aplican a todos los dataset

In [73]:
# Limpieza estándar
def limpieza(dataset):
    # Conserva unicamente los renglones con datos municipales
    dataset = dataset[dataset['Clave geoestadística'].str.len() == 5]
    
    # Elimina columnas que ya no se van a utilizar
    del(dataset['Entidad federativa'])
    del(dataset['Municipio'])
    del(dataset['Localidad'])
    
    # Renombra la columna correspondiente a CVE_MUN y asignala como index
    dataset.set_index('Clave geoestadística', inplace=True)
    dataset = dataset.rename_axis('CVE_MUN')
    
    return dataset

In [64]:
# Metadatos estándar
metadatos = {
    'Nombre del Dataset': 'Características de las localidades y del entorno urbano 2014',
    'Descripcion del dataset': None,
    'Disponibilidad Temporal': '2015',
    'Periodo de actualizacion': 'No Determinada',
    'Nivel de Desagregacion': 'Proyecto (agregable a municipal, estatal y nacional)',
    'Notas': None,
    'Fuente': 'INEGI',
    'URL_Fuente': 'http://www.beta.inegi.org.mx/proyectos/encotras/cleu/2014/',
    'Dataset base': None,
}

In [67]:
list(datasets.keys())

['P0702', 'P0703', 'P0704', 'P0705', 'P0706', 'P0713']

In [78]:
datasets['P0702'].head(3)

,Clave geoestadística,Entidad federativa,Municipio,Localidad,Restricción del paso a automóviles,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
1,00,Estados Unidos Mexicanos,Total,Total,Total,1329936.0,11211.0,48629.0,1228386.0,41710.0
2,00,Estados Unidos Mexicanos,Total,Total,Restricción del paso a automóviles en todas la...,17362.0,9849.0,601.0,6902.0,10.0
3,00,Estados Unidos Mexicanos,Total,Total,Restricción del paso a automóviles en alguna v...,157433.0,286.0,42607.0,114519.0,21.0


In [79]:
# Limpieza estándar inicial
for ds in list(datasets.keys()):
    print('Limpiando dataset para {}'.format(ds))
    datasets[ds] = limpieza(datasets[ds])
    print('------terminado------')

Limpiando dataset para P0702
------terminado------
Limpiando dataset para P0703
------terminado------
Limpiando dataset para P0704
------terminado------
Limpiando dataset para P0705
------terminado------
Limpiando dataset para P0706
------terminado------
Limpiando dataset para P0713
------terminado------


### Limpieza de dataset para P0702: Restricción al paso de automóviles

In [268]:
nom_hojas['P0702']

'Manzanas por entidad federativa, municipio, localidad y restricción del paso a automóviles según restricción del paso a peatones en sus vialidades'

In [93]:
datasets['P0702'].head()

,Restricción del paso a automóviles,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,278.0,553.0,8397.0,191.0
01001,Restricción del paso a automóviles en todas la...,305.0,257.0,5.0,43.0,0.0
01001,Restricción del paso a automóviles en alguna v...,1079.0,4.0,504.0,571.0,0.0
01001,Restricción del paso a automóviles en ninguna ...,7599.0,5.0,40.0,7554.0,0.0
01001,No especificado,436.0,12.0,4.0,229.0,191.0


In [164]:
# La columna 'Restricción del paso a automóviles' clasifica las manzanas
# que tienen alguna clase de restriccion al paso de automóviles.
P0702_raw = datasets['P0702']
list(P0702_raw['Restricción del paso a automóviles'].unique())

['Total',
 'Restricción del paso a automóviles en todas las vialidades',
 'Restricción del paso a automóviles en alguna vialidad',
 'Restricción del paso a automóviles en ninguna vialidad',
 'No especificado']

In [165]:
P0702_raw.head()

,Restricción del paso a automóviles,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,278.0,553.0,8397.0,191.0
01001,Restricción del paso a automóviles en todas la...,305.0,257.0,5.0,43.0,0.0
01001,Restricción del paso a automóviles en alguna v...,1079.0,4.0,504.0,571.0,0.0
01001,Restricción del paso a automóviles en ninguna ...,7599.0,5.0,40.0,7554.0,0.0
01001,No especificado,436.0,12.0,4.0,229.0,191.0


In [166]:
list(P0702_raw)

['Restricción del paso a automóviles',
 'Total de manzanas',
 'RPPeatones Todas las vialidades',
 'RPPeatones Alguna vialidad',
 'RPPeatones Ninguna vialidad',
 'RPPeatones No especificado']

In [167]:
# Es conveniente crear un dataset con el total de manzanas por municipio
manzanas = P0702_raw[P0702_raw['Restricción del paso a automóviles'] == 'Total']['Total de manzanas']
manzanas.head()

CVE_MUN
01001    9419.0
01002     140.0
01003     585.0
01004     144.0
01005    1195.0
Name: Total de manzanas, dtype: float64

In [168]:
# Para el parametro, hay que filtrar unicamente las columnas que contienen restricciones a la vialidad
P0702_1 = P0702_raw.loc[P0702_raw['Restricción del paso a automóviles'].isin([
    'Restricción del paso a automóviles en todas las vialidades',
    'Restricción del paso a automóviles en alguna vialidad',
])]
P0702_1.head()

,Restricción del paso a automóviles,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
CVE_MUN,,,,,,
01001,Restricción del paso a automóviles en todas la...,305.0,257.0,5.0,43.0,0.0
01001,Restricción del paso a automóviles en alguna v...,1079.0,4.0,504.0,571.0,0.0
01002,Restricción del paso a automóviles en alguna v...,6.0,0.0,2.0,4.0,0.0
01003,Restricción del paso a automóviles en todas la...,1.0,0.0,0.0,1.0,0.0
01003,Restricción del paso a automóviles en alguna v...,114.0,0.0,9.0,105.0,0.0


In [169]:
# Una vez filtrados hay que agregar los datos por municipio. También vale la pena quitar las columnas de 
# restriccion al paso de peatones, pues no son de interes para este parametro.
P0702_1 = P0702_1['Total de manzanas'].groupby(level=0).sum().to_frame(name='Restricción del paso a automóviles')
P0702_1.head()

,Restricción del paso a automóviles
CVE_MUN,
01001,1384.0
01002,6.0
01003,115.0
01004,1.0
01005,132.0


In [170]:
# Unimos ambos datasets para crear el dataset estandar del parámetro
P0702 = manzanas.to_frame().join(P0702_1)
print(len(P0702))
P0702.head()

1798


,Total de manzanas,Restricción del paso a automóviles
CVE_MUN,,
01001,9419.0,1384.0
01002,140.0,6.0
01003,585.0,115.0
01004,144.0,1.0
01005,1195.0,132.0


Después de hacer el join de las columnas del total de manzanas y las manzanas con restricción al paso de automóviles, en algunas celdas de la columna "Restricción del paso a automóviles" quedaron sin valores. Para entender a qué se debe, es conveniente agregar las columnas de 'Restricción del paso a automóviles en ninguna vialidad' y 'No Especificado'.

In [171]:
# celdas de la columna "Restricción del paso a automóviles" que quedaron sin valores
P0702[P0702['Restricción del paso a automóviles'].isnull()]

,Total de manzanas,Restricción del paso a automóviles
CVE_MUN,,
04007,86.0,NaN
05023,190.0,NaN
05036,81.0,NaN
05037,357.0,NaN
06006,99.0,NaN
07016,149.0,NaN
07023,77.0,NaN
07025,71.0,NaN
07026,97.0,NaN


In [172]:
P0702_raw['Restricción del paso a automóviles'].unique()

array(['Total',
       'Restricción del paso a automóviles en todas las vialidades',
       'Restricción del paso a automóviles en alguna vialidad',
       'Restricción del paso a automóviles en ninguna vialidad',
       'No especificado'], dtype=object)

In [173]:
# Columna con manzanas con restriccion al paso de automoviles en ninguna vialidad
RPA = 'Restricción del paso a automóviles'
TM = 'Total de manzanas'
VAR = 'Restricción del paso a automóviles en ninguna vialidad'
RPANV = P0702_raw[P0702_raw[RPA] == VAR][TM].to_frame(name = VAR)
RPANV.head()

,Restricción del paso a automóviles en ninguna vialidad
CVE_MUN,
01001,7599.0
01002,134.0
01003,470.0
01004,143.0
01005,1031.0


In [174]:
# Columna con manzanas con restriccion al paso de automoviles no especificado
VAR = 'No especificado'
RPANE = P0702_raw[P0702_raw[RPA] == VAR][TM].to_frame(name = 'Restricción del paso a automóviles '+VAR)
RPANE.head()

,Restricción del paso a automóviles No especificado
CVE_MUN,
01001,436.0
01005,32.0
01006,21.0
01007,14.0
01009,2.0


In [175]:
# Union de los dataframes
P0702 = P0702.join(RPANV)
P0702 = P0702.join(RPANE)
P0702.head()

,Total de manzanas,Restricción del paso a automóviles,Restricción del paso a automóviles en ninguna vialidad,Restricción del paso a automóviles No especificado
CVE_MUN,,,,
01001,9419.0,1384.0,7599.0,436.0
01002,140.0,6.0,134.0,NaN
01003,585.0,115.0,470.0,NaN
01004,144.0,1.0,143.0,NaN
01005,1195.0,132.0,1031.0,32.0


Nuevamente, revisamos las celdas que no tienen datos en la columna de "Restriccion al paso de automóviles"

In [197]:
# Celdas de la columna "Restricción del paso a automóviles" que quedaron sin valores
testframe = P0702[P0702['Restricción del paso a automóviles'].isnull()].head()
testframe

,Total de manzanas,Restricción del paso a automóviles,Restricción del paso a automóviles en ninguna vialidad,Restricción del paso a automóviles No especificado
CVE_MUN,,,,
04007,86.0,NaN,86.0,NaN
05023,190.0,NaN,190.0,NaN
05036,81.0,NaN,81.0,NaN
05037,357.0,NaN,337.0,20.0
06006,99.0,NaN,99.0,NaN


Hay dos tipos de casos: 
1. Renglones en donde la columna "Total de manzanas" es igual al numero de manzanas en la columna "Restricción del paso a automóviles en ninguna vialidad" (Por ejemplo, la CVE_MUN 04007). En estos renglones es seguro asumir que el valor de la columna "Restricción del paso a automóviles" es igual a cero.
2. Renglones en donde la columna "Total de manzanas" no es igual al numero de manzanas en la columna "Restricción del paso a automóviles en ninguna vialidad", y la hay una cantidad definida de manzanas en la columna "Restricción del paso a automóviles No especificado". (Por ejemplo, la CVE_MUN 050307) En estos renglones no se puede definir con certeza el valor de la columna "Restricción del paso a automóviles", por lo que se dejará vacío

In [198]:
# Funcion para tratar los casos vacios:
def llenarestriccion(x):
    tm = 'Total de manzanas'
    rpanv = 'Restricción del paso a automóviles en ninguna vialidad'
    if x[tm] == x[rpanv]:
        return 0
    else:
        return x['Restricción del paso a automóviles']

In [196]:
# Corrida de prueba
testframe.apply(llenarestriccion, axis = 1)

CVE_MUN
04007    0.0
05023    0.0
05036    0.0
05037    NaN
06006    0.0
dtype: float64

In [200]:
# Corrida sobre el dataset del paramentro
P0702['Restricción del paso a automóviles'] = P0702.apply(llenarestriccion, axis = 1)

In [215]:
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Manzanas por municipio y restricción del paso a automóviles'
metadatos['Notas'] = 'Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0702meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0702meta.columns = ['Descripcion']
P0702meta = P0702meta.rename_axis('Metadato')
P0702meta

,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Manzanas por municipio y restricción del paso ...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,"Se presentan los resultados de 3,993 localidad..."
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [212]:
# Descripciones de columnas
variables = {
    'Total de manzanas': 'Total de manzanas',
    'Restricción del paso a automóviles': 'Total de manzanas que cuentan con restriccion al paso de automoviles en una o más de sus vialidades',
    'Restricción del paso a automóviles en ninguna vialidad': 'Total de manzanas que no tienen restriccion al paso de automoviles en ninguna de sus vialidades',
    'Restricción del paso a automóviles No especificado': 'Total de manzanas en donde no se especifican las restricciones al paso de automoviles'
}

# Convertir descripciones a dataframe
P0702vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0702vars.columns = ['Descripcion']
P0702vars = P0702vars.rename_axis('Mnemonico')
P0702vars.head()

,Descripcion
Mnemonico,
Total de manzanas,Total de manzanas
Restricción del paso a automóviles,Total de manzanas que cuentan con restriccion ...
Restricción del paso a automóviles en ninguna vialidad,Total de manzanas que no tienen restriccion al...
Restricción del paso a automóviles No especificado,Total de manzanas en donde no se esspecifican ...


In [216]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0702.xlsx'
writer = pd.ExcelWriter(file)
P0702.to_excel(writer, sheet_name = 'DATOS')
P0702meta.to_excel(writer, sheet_name = 'METADATOS')
P0702vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Limpieza de dataset para P0703: Restricción al paso a peatones

In [321]:
nom_hojas['P0703']

'Manzanas por entidad federativa, municipio, localidad y restricción del paso a automóviles según restricción del paso a peatones en sus vialidades'

In [228]:
P0703_raw = datasets['P0703']
P0703_raw.head()

,Restricción del paso a automóviles,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,278.0,553.0,8397.0,191.0
01001,Restricción del paso a automóviles en todas la...,305.0,257.0,5.0,43.0,0.0
01001,Restricción del paso a automóviles en alguna v...,1079.0,4.0,504.0,571.0,0.0
01001,Restricción del paso a automóviles en ninguna ...,7599.0,5.0,40.0,7554.0,0.0
01001,No especificado,436.0,12.0,4.0,229.0,191.0


In [237]:
# Solo nos interesan los renglones con totales
P0703 = P0703_raw[P0703_raw['Restricción del paso a automóviles'] == 'Total']

# Y podemos quitar la columna "Restricción del paso a automóviles"
del(P0703['Restricción del paso a automóviles'])

P0703.head()

,Total de manzanas,RPPeatones Todas las vialidades,RPPeatones Alguna vialidad,RPPeatones Ninguna vialidad,RPPeatones No especificado
CVE_MUN,,,,,
01001,9419.0,278.0,553.0,8397.0,191.0
01002,140.0,0.0,2.0,138.0,0.0
01003,585.0,0.0,11.0,574.0,0.0
01004,144.0,0.0,0.0,144.0,0.0
01005,1195.0,10.0,57.0,1097.0,31.0


In [258]:
# Y pos ese ya es el dataset para el parámetro, nomás faltan los metadatos y las variables.
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Manzanas por municipio y restricción del paso a peatones'
metadatos['Notas'] = 'Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0703meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0703meta.columns = ['Descripcion']
P0703meta = P0703meta.rename_axis('Metadato')
P0703meta

,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Manzanas por municipio y restricción del paso ...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,"Se presentan los resultados de 3,993 localidad..."
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [238]:
# Qué variables tiene este dataset?
list(P0703)

['Total de manzanas',
 'RPPeatones Todas las vialidades',
 'RPPeatones Alguna vialidad',
 'RPPeatones Ninguna vialidad',
 'RPPeatones No especificado']

In [259]:
# Descripciones de columnas
variables = {
    'Total de manzanas': 'Total de manzanas',
    'RPPeatones Todas las vialidades': 'Manzanas con restriccion al paso de peatones en todas las vialidades',
    'RPPeatones Alguna vialidad': 'Manzanas con restriccion al paso de peatones en alguna vialidad',
    'RPPeatones Ninguna vialidad': 'Manzanas con restriccion al paso de peatones en ninguna vialidad',
    'RPPeatones No especificado': 'Manzanas sin especificacion de restriccion al paso de peatones'
}

# Convertir descripciones a dataframe
P0703vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0703vars.columns = ['Descripcion']
P0703vars = P0703vars.rename_axis('Mnemonico')
P0703vars

,Descripcion
Mnemonico,
Total de manzanas,Total de manzanas
RPPeatones Todas las vialidades,Manzanas con restriccion al paso de peatones e...
RPPeatones Alguna vialidad,Manzanas con restriccion al paso de peatones e...
RPPeatones Ninguna vialidad,Manzanas con restriccion al paso de peatones e...
RPPeatones No especificado,Manzanas sin especificacion de restriccion al ...


In [264]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0703.xlsx'
writer = pd.ExcelWriter(file)
P0703.to_excel(writer, sheet_name = 'DATOS')
P0703meta.to_excel(writer, sheet_name = 'METADATOS')
P0703vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Limpieza de dataset para P0704: Vialidades con recubrimiento de la calle

In [256]:
nom_hojas['P0704']

'Vialidades por entidad federativa, municipio, localidad y alumbrado público según tipo de recubrimiento de la calle'

In [246]:
P0704_raw = datasets['P0704']
P0704_raw.head()

,Disponibilidad de alumbrado público,Total de vialidades,Pavimento o concreto,Empedrado o adoquín,Sin recubrimiento,Recubrimiento No especificado
CVE_MUN,,,,,,
01001,Total,44651.0,35050.0,3247.0,3036.0,3318.0
01001,Dispone de alumbrado público,38286.0,33208.0,3019.0,1969.0,90.0
01001,No dispone de alumbrado público,3116.0,1822.0,226.0,1064.0,4.0
01001,No especificado,3249.0,20.0,2.0,3.0,3224.0
01002,Total,669.0,282.0,53.0,311.0,23.0


In [253]:
# Solo nos interesan los renglones con totales
P0704 = P0704_raw[P0704_raw['Disponibilidad de alumbrado público'] == 'Total']

# Y podemos quitar la columna "Disponibilidad de alumbrado público"
del(P0704['Disponibilidad de alumbrado público'])
P0704.head()

,Total de vialidades,Pavimento o concreto,Empedrado o adoquín,Sin recubrimiento,Recubrimiento No especificado
CVE_MUN,,,,,
01001,44651.0,35050.0,3247.0,3036.0,3318.0
01002,669.0,282.0,53.0,311.0,23.0
01003,3500.0,1032.0,1431.0,829.0,208.0
01004,672.0,412.0,5.0,238.0,17.0
01005,6314.0,3991.0,355.0,1392.0,576.0


In [261]:
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Vialidades por municipio según tipo de recubrimiento de la calle'
metadatos['Notas'] = 'Total de vialidades que circundan cada manzana. Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0704meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0704meta.columns = ['Descripcion']
P0704meta = P0704meta.rename_axis('Metadato')
P0704meta

,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Vialidades por municipio según tipo de recubri...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,Total de vialidades que circundan cada manzana...
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [254]:
# Que Variables tiene este dataset?
list(P0704)

['Total de vialidades',
 'Pavimento o concreto',
 'Empedrado o adoquín',
 'Sin recubrimiento',
 'Recubrimiento No especificado']

In [262]:
# Descripciones de columnas
variables = {
    'Total de vialidades': 'Total de Vialidades',
    'Pavimento o concreto': 'Vialidades que cuentan con Pavimento o concreto',
    'Empedrado o adoquín': 'Vialidades que cuentan con Empedrado o adoquín',
    'Sin recubrimiento': 'Vialidades Sin recubrimiento',
    'Recubrimiento No especificado': 'Vialidades con Recubrimiento No especificado'
}

# Convertir descripciones a dataframe
P0704vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0704vars.columns = ['Descripcion']
P0704vars = P0704vars.rename_axis('Mnemonico')
P0704vars

,Descripcion
Mnemonico,
Total de vialidades,Total de Vialidades
Pavimento o concreto,Vialidades que cuentan con Pavimento o concreto
Empedrado o adoquín,Vialidades que cuentan con Empedrado o adoquín
Sin recubrimiento,Vialidades Sin recubrimiento
Recubrimiento No especificado,Vialidades con Recubrimiento No especificado


In [263]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0704.xlsx'
writer = pd.ExcelWriter(file)
P0704.to_excel(writer, sheet_name = 'DATOS')
P0704meta.to_excel(writer, sheet_name = 'METADATOS')
P0704vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Limpieza de dataset para P0705: Vialidades con banqueta

In [269]:
nom_hojas['P0705']

'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de recubrimiento de la calle'

In [270]:
# Dataset base
P0705_raw = datasets['P0705']
P0705_raw.head()

,Disponibilidad de banqueta,Total de manzanas,Banqueta en todas las vialidades,Banqueta en Alguna vialidad,Banqueta en Ninguna vialidad,Banqueta No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,6963.0,1893.0,301.0,262.0
01001,Banqueta en todas las vialidades,6609.0,6269.0,298.0,38.0,4.0
01001,Banqueta en alguna vialidad,2166.0,615.0,1494.0,55.0,2.0
01001,Banqueta en ninguna vialidad,388.0,79.0,101.0,208.0,0.0
01001,No especificado,256.0,0.0,0.0,0.0,256.0


In [293]:
db = 'Disponibilidad de banqueta'
list(P0705_raw[db].unique())

['Total',
 'Banqueta en todas las vialidades',
 'Banqueta en alguna vialidad',
 'Banqueta en ninguna vialidad',
 'No especificado']

In [298]:
# Comenzamos creando un dataset con el total de manzanas por municipio
tm = 'Total de manzanas'
manzanas = P0705_raw[P0705_raw[db] == 'Total'][tm].to_frame()
manzanas.head()

,Total de manzanas
CVE_MUN,
01001,9419.0
01002,140.0
01003,585.0
01004,144.0
01005,1195.0


In [299]:
# Para el parametro, hay que separar la columna "Disponibilidad de banqueta" en una columna por cada subset
tv = 'Banqueta en todas las vialidades'
av = 'Banqueta en alguna vialidad'
nv = 'Banqueta en ninguna vialidad'
ne = 'No especificado'
todasvial = P0705_raw[P0705_raw[db] == tv][tm].to_frame(name = tv)
alguvial = P0705_raw[P0705_raw[db] == av][tm].to_frame(name = av)
ningvial = P0705_raw[P0705_raw[db] == nv][tm].to_frame(name = nv)
nevial = P0705_raw[P0705_raw[db] == ne][tm].to_frame(name = ne)

In [301]:
# Después es necesario unir cada serie a partir del dataset 'manzanas'
P0705 = manzanas.join(todasvial).join(alguvial).join(ningvial).join(nevial)
P0705.head()

,Total de manzanas,Banqueta en todas las vialidades,Banqueta en alguna vialidad,Banqueta en ninguna vialidad,No especificado
CVE_MUN,,,,,
01001,9419.0,6609.0,2166.0,388.0,256.0
01002,140.0,32.0,60.0,48.0,NaN
01003,585.0,186.0,302.0,97.0,NaN
01004,144.0,75.0,48.0,21.0,NaN
01005,1195.0,574.0,431.0,159.0,31.0


In [308]:
# celdas de la columna "Banqueta en todas las vialidades" que quedaron sin valores
P0705[P0705['Banqueta en todas las vialidades'].isnull()].tail()

,Total de manzanas,Banqueta en todas las vialidades,Banqueta en alguna vialidad,Banqueta en ninguna vialidad,No especificado
CVE_MUN,,,,,
30098,12.0,NaN,NaN,12.0,NaN
30107,13.0,NaN,5.0,8.0,NaN
31036,122.0,NaN,30.0,91.0,1.0
31053,279.0,NaN,59.0,209.0,11.0
31105,92.0,NaN,32.0,60.0,NaN


Para los renglones en donde el valor de la columna "Total de Manzanas" es igual al valor de la columna "Banqueta en ninguna vialidad" se puede reemplazar el valor NaN por Cero.

In [311]:
P0705[P0705[tm] == P0705[nv]].head()

,Total de manzanas,Banqueta en todas las vialidades,Banqueta en alguna vialidad,Banqueta en ninguna vialidad,No especificado
CVE_MUN,,,,,
12037,67.0,NaN,NaN,67.0,NaN
12069,53.0,NaN,NaN,53.0,NaN
15124,31.0,NaN,NaN,31.0,NaN
20019,23.0,NaN,NaN,23.0,NaN
20024,33.0,NaN,NaN,33.0,NaN


In [312]:
# Funcion para tratar los casos vacios:
def llenarestriccion(x):
    if x[tm] == x[nv]:
        return 0
    else:
        return x['Banqueta en todas las vialidades']
        # Es seguro utilizar la misma funcion para las columnas 
        # "Banqueta en todas las vialidades" como en la columna
        # "Banqueta en alguna vialidad"

In [315]:
P0705[tv]= P0705.apply(llenarestriccion, axis = 1)
P0705[av]= P0705.apply(llenarestriccion, axis = 1)
P0705.loc['12037']

Total de manzanas                   67.0
Banqueta en todas las vialidades     0.0
Banqueta en alguna vialidad          0.0
Banqueta en ninguna vialidad        67.0
No especificado                      NaN
Name: 12037, dtype: float64

In [317]:
nom_hojas['P0705']

'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de recubrimiento de la calle'

In [318]:
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Manzanas por municipio segun disponibilidad de banqueta en sus vialidades'
metadatos['Notas'] = 'Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0705meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0705meta.columns = ['Descripcion']
P0705meta = P0705meta.rename_axis('Metadato')
P0705meta


,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Manzanas por municipio segun disponibilidad de...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,"Se presentan los resultados de 3,993 localidad..."
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [316]:
list(P0705)

['Total de manzanas',
 'Banqueta en todas las vialidades',
 'Banqueta en alguna vialidad',
 'Banqueta en ninguna vialidad',
 'No especificado']

In [319]:
# Descripciones de columnas
variables = {
    'Total de manzanas': 'Total de manzanas',
    'Banqueta en todas las vialidades': 'Manzanas que cuentan con banqueta en todas las vialidades',
    'Banqueta en alguna vialidad': 'Manzanas que cuentan con banqueta en alguna vialidad',
    'Banqueta en ninguna vialidad': 'Manzanas que cuentan con banqueta en ninguna vialidad',
    'No especificado': 'Manzanas en donde no se ha especificado si cuenta o no con banqueta'
}

P0705vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0705vars.columns = ['Descripcion']
P0705vars = P0705vars.rename_axis('Mnemonico')
P0705vars

,Descripcion
Mnemonico,
Total de manzanas,Total de manzanas
Banqueta en todas las vialidades,Manzanas que cuentan con banqueta en todas las...
Banqueta en alguna vialidad,Manzanas que cuentan con banqueta en alguna vi...
Banqueta en ninguna vialidad,Manzanas que cuentan con banqueta en ninguna v...
No especificado,Manzanas en donde no se ha especificado si cue...


In [320]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0705.xlsx'
writer = pd.ExcelWriter(file)
P0705.to_excel(writer, sheet_name = 'DATOS')
P0705meta.to_excel(writer, sheet_name = 'METADATOS')
P0705vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Limpieza de dataset para P0706: Vialidades con guarnición

In [271]:
nom_hojas['P0706']

'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de guarnición en sus vialidades'

In [272]:
# Dataset base
P0706_raw = datasets['P0706']
P0706_raw.head()

,Disponibilidad de banqueta,Total de manzanas,Guarnicion en Todas las vialidades,Guarnicion en Alguna vialidad,Guarnicion en Ninguna vialidad,Guarnicion No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,6448.0,2356.0,359.0,256.0
01001,Banqueta en todas las vialidades,6609.0,6197.0,346.0,66.0,0.0
01001,Banqueta en alguna vialidad,2166.0,180.0,1952.0,34.0,0.0
01001,Banqueta en ninguna vialidad,388.0,71.0,58.0,259.0,0.0
01001,No especificado,256.0,0.0,0.0,0.0,256.0


In [273]:
list(P0706_raw)

['Disponibilidad de banqueta',
 'Total de manzanas',
 'Guarnicion en Todas las vialidades',
 'Guarnicion en Alguna vialidad',
 'Guarnicion en Ninguna vialidad',
 'Guarnicion No especificado']

In [274]:
# Solo nos interesan los renglones con totales
P0706 = P0706_raw[P0706_raw['Disponibilidad de banqueta'] == 'Total']

# Y podemos quitar la columna "Disponibilidad de alumbrado público"
del(P0706['Disponibilidad de banqueta'])
P0706.head()

,Total de manzanas,Guarnicion en Todas las vialidades,Guarnicion en Alguna vialidad,Guarnicion en Ninguna vialidad,Guarnicion No especificado
CVE_MUN,,,,,
01001,9419.0,6448.0,2356.0,359.0,256.0
01002,140.0,32.0,61.0,47.0,0.0
01003,585.0,219.0,283.0,83.0,0.0
01004,144.0,74.0,50.0,20.0,0.0
01005,1195.0,663.0,403.0,98.0,31.0


In [275]:
hojas

{'P0702': 'TEU07',
 'P0703': 'TEU07',
 'P0704': 'TEU12',
 'P0705': 'TEU02',
 'P0706': 'TEU03',
 'P0713': 'TEU04'}

In [276]:
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Manzanas por municipio segun disponibilidad de guarnición en sus vialidades'
metadatos['Notas'] = 'Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0706meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0706meta.columns = ['Descripcion']
P0706meta = P0706meta.rename_axis('Metadato')
P0706meta

,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Manzanas por municipio segun disponibilidad de...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,"Se presentan los resultados de 3,993 localidad..."
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [277]:
# Que Variables tiene este dataset?
list(P0706)

['Total de manzanas',
 'Guarnicion en Todas las vialidades',
 'Guarnicion en Alguna vialidad',
 'Guarnicion en Ninguna vialidad',
 'Guarnicion No especificado']

In [287]:
# Descripciones de columnas
variables = {
    'Total de manzanas': 'Total de manzanas',
    'Guarnicion en Todas las vialidades': 'Manzanas con guarnicion en Todas las vialidades',
    'Guarnicion en Alguna vialidad': 'Manzanas con guarnicion en Alguna vialidad',
    'Guarnicion en Ninguna vialidad': 'Manzanas con guarnicion en Ninguna vialidad',
    'Guarnicion No especificado': 'Manzanas en donde no se ha especificado si cuenta o no con guarnicion'
}

P0706vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0706vars.columns = ['Descripcion']
P0706vars = P0706vars.rename_axis('Mnemonico')
P0706vars

,Descripcion
Mnemonico,
Total de manzanas,Total de manzanas
Guarnicion en Todas las vialidades,Manzanas con guarnicion en Todas las vialidades
Guarnicion en Alguna vialidad,Manzanas con guarnicion en Alguna vialidad
Guarnicion en Ninguna vialidad,Manzanas con guarnicion en Ninguna vialidad
Guarnicion No especificado,Manzanas en donde no se ha especificado si cue...


In [288]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0706.xlsx'
writer = pd.ExcelWriter(file)
P0706.to_excel(writer, sheet_name = 'DATOS')
P0706meta.to_excel(writer, sheet_name = 'METADATOS')
P0706vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')

---------------TERMINADO---------------


### Limpieza de dataset para P0713: Vialidades con rampa para silla de ruedas

In [280]:
nom_hojas['P0713']

'Manzanas por entidad federativa, municipio, localidad y disponibilidad de banqueta según disponibilidad de rampa para silla de ruedas en sus vialidades'

In [281]:
# Dataset base
P0713_raw = datasets['P0713']
P0713_raw.head()

,Disponibilidad de banqueta,Total de manzanas,Rampa en Todas las vialidades,Rampa en Alguna vialidad,Rampa en Ninguna vialidad,Rampa No especificado
CVE_MUN,,,,,,
01001,Total,9419.0,1920.0,2741.0,4502.0,256.0
01001,Banqueta en todas las vialidades,6609.0,1919.0,1624.0,3066.0,0.0
01001,Banqueta en alguna vialidad,2166.0,1.0,1117.0,1048.0,0.0
01001,Banqueta en ninguna vialidad,388.0,0.0,0.0,388.0,0.0
01001,No especificado,256.0,0.0,0.0,0.0,256.0


In [282]:
list(P0713_raw)

['Disponibilidad de banqueta',
 'Total de manzanas',
 'Rampa en Todas las vialidades',
 'Rampa en Alguna vialidad',
 'Rampa en Ninguna vialidad',
 'Rampa No especificado']

In [283]:
# Solo nos interesan los renglones con totales
P0713 = P0713_raw[P0713_raw['Disponibilidad de banqueta'] == 'Total']

# Y podemos quitar la columna "Disponibilidad de alumbrado público"
del(P0713['Disponibilidad de banqueta'])
P0713.head()

,Total de manzanas,Rampa en Todas las vialidades,Rampa en Alguna vialidad,Rampa en Ninguna vialidad,Rampa No especificado
CVE_MUN,,,,,
01001,9419.0,1920.0,2741.0,4502.0,256.0
01002,140.0,0.0,5.0,135.0,0.0
01003,585.0,3.0,36.0,545.0,1.0
01004,144.0,3.0,18.0,123.0,0.0
01005,1195.0,272.0,323.0,569.0,31.0


In [284]:
metadatos

{'Dataset base': None,
 'Descripcion del dataset': 'Manzanas por municipio segun disponibilidad de guarnición en sus vialidades',
 'Disponibilidad Temporal': '2014',
 'Fuente': 'INEGI',
 'Nivel de Desagregacion': 'Localidad, Municipal, Estatal, Nacional',
 'Nombre del Dataset': 'Características de las localidades y del entorno urbano 2014',
 'Notas': 'Se presentan los resultados de 3,993 localidades con información de entorno urbano.\xa0Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.',
 'Periodo de actualizacion': 'No Determinada',
 'URL_Fuente': 'http://www.beta.inegi.org.mx/proyectos/encotras/cleu/2014/'}

In [285]:
# metadatos especificos para este parametro
metadatos['Descripcion del dataset'] = 'Manzanas municipio según disponibilidad de rampa para silla de ruedas en sus vialidades'
metadatos['Notas'] = 'Total de vialidades que circundan cada manzana. Se presentan los resultados de 3,993 localidades con información de entorno urbano. Se excluyen 35,839 manzanas que corresponden a edificios de conjuntos habitacionales.'

# Convertir metadatos a dataframe
P0713meta = pd.DataFrame.from_dict(metadatos, orient='index', dtype=None)
P0713meta.columns = ['Descripcion']
P0713meta = P0713meta.rename_axis('Metadato')
P0713meta

,Descripcion
Metadato,
Nombre del Dataset,Características de las localidades y del entor...
Descripcion del dataset,Manzanas municipio según disponibilidad de ram...
Disponibilidad Temporal,2014
Periodo de actualizacion,No Determinada
Nivel de Desagregacion,"Localidad, Municipal, Estatal, Nacional"
Notas,Total de vialidades que circundan cada manzana...
Fuente,INEGI
URL_Fuente,http://www.beta.inegi.org.mx/proyectos/encotra...
Dataset base,None


In [286]:
# Que Variables tiene este dataset?
list(P0713)

['Total de manzanas',
 'Rampa en Todas las vialidades',
 'Rampa en Alguna vialidad',
 'Rampa en Ninguna vialidad',
 'Rampa No especificado']

In [289]:
# Descripciones de columnas
variables = {
    'Total de manzanas': 'Total de manzanas',
    'Rampa en Todas las vialidades': 'Manzanas con rampa en Todas las vialidades',
    'Rampa en Alguna vialidad': 'Manzanas con rampa en Alguna vialidad',
    'Rampa en Ninguna vialidad': 'Manzanas con rampa en Ninguna vialidad',
    'Rampa No especificado': 'Manzanas en donde no se ha especificado si cuenta o no con rampa'
}

P0713vars = pd.DataFrame.from_dict(variables, orient='index', dtype=None)
P0713vars.columns = ['Descripcion']
P0713vars = P0713vars.rename_axis('Mnemonico')
P0713vars

,Descripcion
Mnemonico,
Total de manzanas,Total de manzanas
Rampa en Todas las vialidades,Manzanas con rampa en Todas las vialidades
Rampa en Alguna vialidad,Manzanas con rampa en Alguna vialidad
Rampa en Ninguna vialidad,Manzanas con rampa en Ninguna vialidad
Rampa No especificado,Manzanas en donde no se ha especificado si cue...


In [290]:
# Guardar el dataset
file = r'D:\PCCS\01_Dmine\Datasets\CLEU\P0713.xlsx'
writer = pd.ExcelWriter(file)
P0713.to_excel(writer, sheet_name = 'DATOS')
P0713meta.to_excel(writer, sheet_name = 'METADATOS')
P0713vars.to_excel(writer, sheet_name = 'VARIABLES')
writer.save()
print('---------------TERMINADO---------------')


---------------TERMINADO---------------
